In [23]:
#trying this tutorial: https://www.natekratzer.com/posts/census_map/
import numpy as np
import geopandas as gpd
import requests

In [24]:
#reading in file and mapping counties using WI as example
county_shp = gpd.read_file('cb_2022_us_county_500k.zip')
CA_counties = county_shp[county_shp.STATE_NAME == 'California']

In [ ]:
#wi_counties.explore()

In [26]:
#load API key
with open('/Users/mafphd/datathon/datathon-venv/census_api_key.txt') as key:
    api_key=key.read().strip()

#specify the data source by year and survey
year = '2020'
dsource = 'dec' #Decennial Census
dname = 'dhc' #Demographic and Housing Characteristics
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

#unique to this specific data request
cols = 'NAME,P1_001N' #NAME of geography as well as the variables I want to pull
geo = 'county:*' #county geography level
state = 'state:*' #all states

#add unique request features to the base_url
data_url = f'{base_url}?get={cols}&for={geo}&in={state}&key={api_key}'

#go get the data
response = requests.get(data_url)



In [27]:
#take the data in json and format it into a dataframe
data = response.json()

In [28]:
df = pd.DataFrame(data = data[1:], columns = data[0])
df.head()

,NAME,P1_001N,state,county
0,"Madera County, California",156255,06,039
1,"Marin County, California",262321,06,041
2,"Mariposa County, California",17131,06,043
3,"Mendocino County, California",91601,06,045
4,"Merced County, California",281202,06,047


In [29]:
#rename columns and convert population to numeric
df.rename(columns = {'NAME': 'county_name', 'P1_001N': 'population', 'state': 'state_fips', 'county': 'county_fips'}, inplace=True)
df['population'] = df['population'].apply(pd.to_numeric)

In [30]:
df['fips'] = df['state_fips'] + df['county_fips']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   county_name  3221 non-null   object
 1   population   3221 non-null   int64 
 2   state_fips   3221 non-null   object
 3   county_fips  3221 non-null   object
 4   fips         3221 non-null   object
dtypes: int64(1), object(4)
memory usage: 125.9+ KB


In [ ]:
#states = county_shp.STATE_NAME.values.tolist()
#print(states)

In [ ]:
#creating a full US map
#usa_counties = county_shp[county_shp.STATE_NAME == states]


In [ ]:
#usa_counties.explore()

In [ ]:
#construct the fips code to match with the geography data
#usa_counties['fips'] = usa_counties['STATEFP'] + usa_counties['COUNTYFP']
#usa_counties.head()

In [32]:
#merge in the new data
#clean up the names to be a bit more presentable
#df_map = (CA_counties.merge(df, how = 'left', on = 'fips').rename(columns = {'NAMELSAD': 'County Name', 'population': 'County Population'}))

#df_map.head()

In [ ]:
#df_map.explore(
    #column ='County Population',
    #scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
    #legend=True,  # show legend
    #k=5,  # use 5 bins
    #tooltip = ['County Name', 'County Population'],
    #tiles = 'CartoDB positron', #fades into the background better than the default
    #legend_kwds=dict(colorbar=False, color="gray"))

In [ ]:
#biggest_counties = df_map[df_map['County Population'] > 5000000]
#biggest_counties

In [ ]:
#another approach I found, but did not work well bc of size of dataset and rate limits
#pip install geopy

In [ ]:
#note: rate limit became a problem; will need to filter down to CA data only before retrying
#from geopy.geocoders import Nominatim

#def get_zip_code(latitude, longitude): 
    #geolocator = Nominatim(user_agent="mafphd@icloud.com")
    #location = geolocator.reverse((latitude, longitude), exactly_one=True)
    #address = location.raw['address']
    #zip_code = address.get('postcode')
    
    #return zip_code

In [33]:
import pandas as pd
burn = pd.read_csv('Burn_Severity_Trends.csv')
burn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30418 entries, 0 to 30417
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   X                    30418 non-null  float64
 1   Y                    30418 non-null  float64
 2   OBJECTID             30418 non-null  int64  
 3   FIRE_ID              30418 non-null  object 
 4   FIRE_NAME            30418 non-null  object 
 5   ASMNT_TYPE           30418 non-null  object 
 6   FIRE_TYPE            30418 non-null  object 
 7   NODATA_THRESHOLD     30418 non-null  int64  
 8   GREENNESS_THRESHOLD  30418 non-null  int64  
 9   LOW_THRESHOLD        30418 non-null  int64  
 10  MODERATE_THRESHOLD   30418 non-null  int64  
 11  HIGH_THRESHOLD       30418 non-null  int64  
 12  IG_DATE              30418 non-null  object 
 13  LATITUDE             30418 non-null  float64
 14  LONGITUDE            30418 non-null  float64
 15  ACRES                30418 non-null 

In [ ]:
#burn['ZIP_CODE'] = burn.apply(lambda row: get_zip_code(row['LATITUDE'], row['LONGITUDE']), axis=1)

In [10]:
#trying to identify counties by latitude/longitude coordinates using TIGER/Line Shapefile, 2019, nation, U.S., Current County and Equivalent National Shapefile from data.gov
import geopandas as gpd

# Load the shapefile
gdf = gpd.read_file('/Users/mafphd/datathon/datathon-venv/tl_2019_us_county/tl_2019_us_county.shp') #file is too large to commit; download from https://catalog.data.gov/dataset/tiger-line-shapefile-2019-nation-u-s-current-county-and-equivalent-national-shapefile
def get_county_name(latitude, longitude):
    point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([longitude], [latitude]))
    county = gdf[gdf.contains(point.unary_union)]
    return county['NAME'].iloc[0] if not county.empty else None

# Example usage
latitude = 37.7749  # Example latitude
longitude = -122.4194  # Example longitude

county_name = get_county_name(latitude, longitude)
print("County Name:", county_name)


County Name: San Francisco


In [11]:
burn['County'] = burn.apply(lambda row: get_county_name(row['LATITUDE'], row['LONGITUDE']), axis=1)

In [12]:
burn['County'].isna().sum()

23

In [13]:
#looking at the missing counties to see if any are within approximate CA coordinates (from ChatGPT/verified on a map: min_lat = 32.5, max_lat = 37.0,min_lon = -125.0,max_lon = -114.0)
missing_ctys = burn[burn['County'].isna()]
missing_ctys_sorted = missing_ctys[['LATITUDE', 'LONGITUDE']]
missing_ctys_sorted.sort_values(by='LATITUDE', ascending = True)

,LATITUDE,LONGITUDE
19004,31.299,-110.357
20577,31.312,-108.834
27052,31.318,-108.714
8894,31.329,-108.998
30154,31.345,-111.132
24091,31.346,-111.120
27746,32.554,-116.694
5611,32.559,-116.714
19073,32.579,-116.264
10783,32.585,-116.514


In [14]:
#using BallTree nearest neighbors approach to fill in County based on closest known lat/long coordinates
from sklearn.neighbors import BallTree

# Assuming 'burn' is your original DataFrame

# Create a subset DataFrame with only rows that have County information
known_counties = burn.dropna(subset=['County'])

# Extract latitude and longitude for known counties
known_coordinates = known_counties[['LATITUDE', 'LONGITUDE']].values

# Build a BallTree for efficient nearest neighbor search
tree = BallTree(known_coordinates, leaf_size=15)

# Define a function to impute missing County values based on nearest neighbors
def impute_county(row):
    if pd.isna(row['County']):
        distances, indices = tree.query([[row['LATITUDE'], row['LONGITUDE']]], k=1)
        closest_county = known_counties.iloc[indices[0][0]]['County']
        return closest_county
    else:
        return row['County']

# Apply the impute_county function to fill in missing values
burn['County'] = burn.apply(impute_county, axis=1)


In [15]:
burn['County'].isna().sum()

0

In [16]:
#checking to see if I can get fips codes from the shapefile to join the two dataframes
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   3233 non-null   object  
 1   COUNTYFP  3233 non-null   object  
 2   COUNTYNS  3233 non-null   object  
 3   GEOID     3233 non-null   object  
 4   NAME      3233 non-null   object  
 5   NAMELSAD  3233 non-null   object  
 6   LSAD      3233 non-null   object  
 7   CLASSFP   3233 non-null   object  
 8   MTFCC     3233 non-null   object  
 9   CSAFP     1255 non-null   object  
 10  CBSAFP    1915 non-null   object  
 11  METDIVFP  110 non-null    object  
 12  FUNCSTAT  3233 non-null   object  
 13  ALAND     3233 non-null   int64   
 14  AWATER    3233 non-null   int64   
 15  INTPTLAT  3233 non-null   object  
 16  INTPTLON  3233 non-null   object  
 17  geometry  3233 non-null   geometry
dtypes: geometry(1), int64(2), object(15)
memory usage: 454.8+ KB


In [34]:
# bringing FP codes to the burn dataframe for joining
# Spatial join between burn DataFrame and shapefile
gdf = gdf[['STATEFP', 'COUNTYFP', 'geometry']]  # Select only necessary columns
burn_gdf = gpd.GeoDataFrame(burn, geometry=gpd.points_from_xy(burn.LONGITUDE, burn.LATITUDE), crs=gdf.crs)
merged = gpd.sjoin(burn_gdf, gdf, how='left', op='within')

# Extract STATEFP and COUNTYFP values
burn['state_fips'] = merged['STATEFP']
burn['county_fips'] = merged['COUNTYFP']

# Create 'fips' column by concatenating 'state_fips' and 'county_fips'
burn['fips'] = burn['state_fips'].astype(str) + burn['county_fips'].astype(str)


/Users/mafphd/datathon/datathon-venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [18]:
burn.head()

,X,Y,OBJECTID,FIRE_ID,FIRE_NAME,ASMNT_TYPE,FIRE_TYPE,NODATA_THRESHOLD,GREENNESS_THRESHOLD,LOW_THRESHOLD,...,PERIM_ID,DNBR_OFFST,DNBR_STDDV,COMMENTS,PRE_ID,POST_ID,County,state_fips,county_fips,fips
0,-144.673246,64.482309,830721,AK6448214467319840607,GLACIER CREEK,Initial (SS),Wildfire,9999,9999,550,...,506601519860705,-9999,-9999,NaN,NaN,506801519840713,Southeast Fairbanks,02,240,02240
1,-102.849877,43.083926,830722,SD4308410285019840906,SANDOZ,Initial,Wildfire,-970,-150,220,...,NaN,211,-9999,NaN,503303019840622,503303019840926,Oglala Lakota,46,102,46102
2,-113.457720,48.804345,830723,MT4880411345819840819,NAPI PEAK,Extended,Wildfire,-970,-150,160,...,NaN,-62,-9999,NaN,504102619840614,504102619850719,Glacier,30,035,30035
3,-106.958289,45.336400,830724,MT4533610695819840826,KIRBY,Extended,Wildfire,-970,-150,150,...,NaN,52,-9999,NaN,503602819840814,503602819850817,Big Horn,30,003,30003
4,-106.322839,45.616844,830725,MT4561710632319840918,LAKE CREEK,Initial,Wildfire,-970,-150,-50,...,NaN,-104,-9999,NaN,503602819840814,503602819841017,Rosebud,30,087,30087


In [35]:
# filtering down to California only for both dataframes
#Cali = df_map[df_map['STATE_NAME']=='California']
#Cali.head()

In [36]:
Cali_burn = burn[burn['state_fips'] == '06']
Cali_burn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1986 entries, 23 to 30374
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   X                    1986 non-null   float64
 1   Y                    1986 non-null   float64
 2   OBJECTID             1986 non-null   int64  
 3   FIRE_ID              1986 non-null   object 
 4   FIRE_NAME            1986 non-null   object 
 5   ASMNT_TYPE           1986 non-null   object 
 6   FIRE_TYPE            1986 non-null   object 
 7   NODATA_THRESHOLD     1986 non-null   int64  
 8   GREENNESS_THRESHOLD  1986 non-null   int64  
 9   LOW_THRESHOLD        1986 non-null   int64  
 10  MODERATE_THRESHOLD   1986 non-null   int64  
 11  HIGH_THRESHOLD       1986 non-null   int64  
 12  IG_DATE              1986 non-null   object 
 13  LATITUDE             1986 non-null   float64
 14  LONGITUDE            1986 non-null   float64
 15  ACRES                1986 non-null   int6

In [41]:
burn_pop_merge = Cali_burn.merge(df, how = 'left', on = 'fips')
burn_pop_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   X                    1986 non-null   float64
 1   Y                    1986 non-null   float64
 2   OBJECTID             1986 non-null   int64  
 3   FIRE_ID              1986 non-null   object 
 4   FIRE_NAME            1986 non-null   object 
 5   ASMNT_TYPE           1986 non-null   object 
 6   FIRE_TYPE            1986 non-null   object 
 7   NODATA_THRESHOLD     1986 non-null   int64  
 8   GREENNESS_THRESHOLD  1986 non-null   int64  
 9   LOW_THRESHOLD        1986 non-null   int64  
 10  MODERATE_THRESHOLD   1986 non-null   int64  
 11  HIGH_THRESHOLD       1986 non-null   int64  
 12  IG_DATE              1986 non-null   object 
 13  LATITUDE             1986 non-null   float64
 14  LONGITUDE            1986 non-null   float64
 15  ACRES                1986 non-null   i

In [42]:
burn_pop_merge.columns

Index(['X', 'Y', 'OBJECTID', 'FIRE_ID', 'FIRE_NAME', 'ASMNT_TYPE', 'FIRE_TYPE',
       'NODATA_THRESHOLD', 'GREENNESS_THRESHOLD', 'LOW_THRESHOLD',
       'MODERATE_THRESHOLD', 'HIGH_THRESHOLD', 'IG_DATE', 'LATITUDE',
       'LONGITUDE', 'ACRES', 'IRWINID', 'MAP_ID', 'MAP_PROG', 'PERIM_ID',
       'DNBR_OFFST', 'DNBR_STDDV', 'COMMENTS', 'PRE_ID', 'POST_ID',
       'state_fips_x', 'county_fips_x', 'fips', 'county_name', 'population',
       'state_fips_y', 'county_fips_y'],
      dtype='object')

In [43]:
#dropping extra columns
burn_pop_merge.drop(['state_fips_y', 'county_fips_y'], axis=1, inplace=True)
burn_pop_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1986 entries, 0 to 1985
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   X                    1986 non-null   float64
 1   Y                    1986 non-null   float64
 2   OBJECTID             1986 non-null   int64  
 3   FIRE_ID              1986 non-null   object 
 4   FIRE_NAME            1986 non-null   object 
 5   ASMNT_TYPE           1986 non-null   object 
 6   FIRE_TYPE            1986 non-null   object 
 7   NODATA_THRESHOLD     1986 non-null   int64  
 8   GREENNESS_THRESHOLD  1986 non-null   int64  
 9   LOW_THRESHOLD        1986 non-null   int64  
 10  MODERATE_THRESHOLD   1986 non-null   int64  
 11  HIGH_THRESHOLD       1986 non-null   int64  
 12  IG_DATE              1986 non-null   object 
 13  LATITUDE             1986 non-null   float64
 14  LONGITUDE            1986 non-null   float64
 15  ACRES                1986 non-null   i

In [45]:
burn_pop_merge.rename(columns = {'state_fips_x': 'state_fips', 'county_fips_x': 'county_fips'}, inplace=True)
burn_pop_merge.columns

Index(['X', 'Y', 'OBJECTID', 'FIRE_ID', 'FIRE_NAME', 'ASMNT_TYPE', 'FIRE_TYPE',
       'NODATA_THRESHOLD', 'GREENNESS_THRESHOLD', 'LOW_THRESHOLD',
       'MODERATE_THRESHOLD', 'HIGH_THRESHOLD', 'IG_DATE', 'LATITUDE',
       'LONGITUDE', 'ACRES', 'IRWINID', 'MAP_ID', 'MAP_PROG', 'PERIM_ID',
       'DNBR_OFFST', 'DNBR_STDDV', 'COMMENTS', 'PRE_ID', 'POST_ID',
       'state_fips', 'county_fips', 'fips', 'county_name', 'population'],
      dtype='object')

In [46]:
#writing file to csv
burn_pop_merge.to_csv('burn_severity_population.csv')

In [ ]:
#extras = burn_pop_merge[burn_pop_merge['POST_ID'].isna()]
#print(extras)


In [ ]:
#extras.info()

In [ ]:
#dropping the two extra rows that got added, since they don't seem to have any info from the burn data
#burn_pop_merge = burn_pop_merge.dropna(subset=['POST_ID'])
#burn_pop_merge.info()

In [ ]:
pip install seaborn

In [ ]:
# quick visualization of the results
import matplotlib.pyplot as plt
import seaborn as sns
# Group by 'County Name' and get the count of POST_ID
county_counts = burn_pop_merge.groupby('County Name')['POST_ID'].count()

# Filter for counties with more than 5 POST_ID values
county_counts_filtered = county_counts[county_counts > 5]

# Merge with the original DataFrame to keep 'County Population'
merged_df = burn_pop_merge.merge(county_counts_filtered, left_on='County Name', right_index=True, suffixes=('_original', '_count'))

# Sort counties by County Population in descending order
sorted_df = merged_df.sort_values(by='County Population', ascending=False)
# Create the bar chart
plt.figure(figsize=(20, 6))
sns.barplot(x=sorted_df['County Name'], y=sorted_df['POST_ID_count'],color='firebrick')
plt.title('Counties with > 5 fire records, Sorted by Population')
plt.xlabel('County Name')
plt.ylabel('Fires')
plt.xticks(rotation=90)
plt.show()
